#掛載雲端

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 載入 AOI瑕疵檢測 資料集

In [2]:
#將壓縮檔複製到/content
! cp  "/content/drive/MyDrive/未命名資料夾/train_images.zip" /content/

In [3]:
#解壓縮訓練集
! unzip /content/train_images > data_unzip.log

In [4]:
#read csv
import pandas as pd
AOI_data = pd.read_csv('/content/drive/MyDrive/未命名資料夾/train.csv')

In [5]:
#查看訓練集每類別圖片數量並且分類
label = []
for i in range(6):
  temp = AOI_data[AOI_data['Label'] == i]
  label.append(temp.reset_index())
  print('第' + str(i) + '類張數: ' + str(len(label[i])))


第0類張數: 674
第1類張數: 492
第2類張數: 100
第3類張數: 378
第4類張數: 240
第5類張數: 644


In [6]:
import cv2
train_images = []
train_label = []
test_images = []
test_label = []
for i in range(6):
  #讀取圖片測試集圖片(這邊的程式碼不可更動)
  images_temp = []
  label_temp = [i] * 20
  for j in range(20):
    img = cv2.imread('/content/train_images/'+label[i]['ID'][j])
    images_temp.append(cv2.resize(img,(224,224), cv2.INTER_AREA))
  test_images += images_temp
  test_label += label_temp

  #將剩餘圖片進行增量，使訓練集每類張數相同

  train_images += images_temp * 16
  train_label += label_temp * 16


In [7]:
#將 list 轉成 array
import numpy as np
from sklearn.utils import shuffle
x_train = np.array(train_images)
x_test = np.array(test_images)
y_train = np.array(train_label)
y_test = np.array(test_label)

In [8]:
#將將訓練集進行shuffle
import random
x_train , y_train = shuffle(x_train, y_train, random_state=random.seed())

# 查看資料的 shape

In [9]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (1920, 224, 224, 3)
x_test shape:  (120, 224, 224, 3)


# 查看標籤的 shape

In [10]:
print('y_train shape:',y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (1920,)
y_test shape: (120,)


# 查看標籤的內容 

In [11]:
print(y_train[0:10])

[5 5 2 4 0 1 2 2 5 1]


# 進行 min-max normalization

In [12]:
x_train_norm = x_train.astype('float32')/255
x_test_norm = x_test.astype('float32')/255

In [13]:
print(x_train_norm[0][0][0])

[0.8509804 0.8509804 0.8509804]


# 將數字標籤進行 One-hot 編碼

In [14]:
from tensorflow.keras import utils

# 轉換前
print(y_train[0])

5


In [15]:
# 進行 One-hot 編碼轉換...
y_train_onehot = utils.to_categorical(y_train, 6)
y_test_onehot = utils.to_categorical(y_test, 6)
# 轉換後
print(y_train_onehot[0])

[0. 0. 0. 0. 0. 1.]


# 建立 CNN 神經網路架構

In [16]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import tensorflow.keras.applications as tensorflow_model

In [17]:
vgg16 = tensorflow_model.VGG16() #請輸入參數

553476096/553467096 [==============================] - 22s 0us/step


In [18]:
#修改全聯階層的輸出類別數
num_classes = 6
cnn = Sequential()
cnn.add(Conv2D(32,(5,5),activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(num_classes,activation='softmax'))

In [19]:
# 神經網路的訓練配置 #
adam = tf.optimizers.Adam(0.0001)
cnn.compile(optimizer = adam,loss = 'categorical_crossentropy', metrics = ['acc'])

In [20]:
# 進行訓練 #
history = cnn.fit(x=x_train_norm,y=y_train_onehot,batch_size=32,epochs=20,validation_split=0.1)

Epoch 1/20
54/54 [==============================] - 44s 805ms/step - loss: 2.0428 - acc: 0.2830 - val_loss: 1.3150 - val_acc: 0.4115
Epoch 2/20
54/54 [==============================] - 45s 828ms/step - loss: 1.0740 - acc: 0.6568 - val_loss: 0.8653 - val_acc: 0.8490
Epoch 3/20
54/54 [==============================] - 44s 816ms/step - loss: 0.7385 - acc: 0.8773 - val_loss: 0.6208 - val_acc: 0.9271
Epoch 4/20
54/54 [==============================] - 44s 815ms/step - loss: 0.5622 - acc: 0.9167 - val_loss: 0.5082 - val_acc: 0.9427
Epoch 5/20
54/54 [==============================] - 44s 813ms/step - loss: 0.4206 - acc: 0.9514 - val_loss: 0.3599 - val_acc: 1.0000
Epoch 6/20
54/54 [==============================] - 45s 829ms/step - loss: 0.3267 - acc: 0.9786 - val_loss: 0.2832 - val_acc: 1.0000
Epoch 7/20
54/54 [==============================] - 44s 814ms/step - loss: 0.2631 - acc: 0.9844 - val_loss: 0.2198 - val_acc: 0.9948
Epoch 8/20
54/54 [==============================] - 44s 815ms/step - 

# 使用測試資料評估神經網路

In [21]:
# evaluate
test_loss,test_val = cnn.evaluate(x_test_norm,y_test_onehot)
print('測試資料損失值:', test_loss)
print('測試資料準確度:', test_val)

4/4 [==============================] - 1s 175ms/step - loss: 0.0299 - acc: 1.0000
測試資料損失值: 0.02985437773168087
測試資料準確度: 1.0


In [22]:
# predict
predict_prop = cnn.predict(x_test_norm)
print('第一筆測試資料的預測機率', predict_prop[0])

第一筆測試資料的預測機率 [9.6216881e-01 1.9998010e-04 2.3283752e-02 1.3553067e-02 7.9433376e-04
 4.5850690e-08]


# 將模型轉換成ONNX

In [24]:
#安裝套件 tf2onnx
!pip install tf-estimator-nightly==2.8.0.dev2021122109 #colab環境缺少此套件 需安裝此套件才能完整安裝tf2onnx
!pip install git+https://github.com/onnx/tensorflow-onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/onnx/tensorflow-onnx to /tmp/pip-req-build-3avnj77s
  Running command git clone -q https://github.com/onnx/tensorflow-onnx /tmp/pip-req-build-3avnj77s
     |████████████████████████████████| 12.8 MB 4.3 MB/s 
  Created wheel for tf2onnx: filename=tf2onnx-1.11.0-py3-none-any.whl size=440601 sha256=661c623170a1cf6855e1e3a8bd9fa3577d68e551d9990dad953e3a7cb705b92f
  Stored in directory: /tmp/pip-ephem-wheel-cache-9if1oss6/wheels/73/99/8e/3034c9b47124da4ac24a6ea72b8da1be5af878e37c2e953bb3
Successfully built tf2onnx
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


In [26]:
#進行轉換
import tf2onnx

aoi, external_tensor_storage = tf2onnx.convert.from_keras(cnn,opset=11,inputs_as_nchw=(1,244,244,3))

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [27]:
#檢查opset版本
print(aoi.opset_import)


[domain: ""
version: 11
, domain: "ai.onnx.ml"
version: 2
]


In [29]:
#由於耐能的量化過程只支持opset ai.onnx 所以移除ai.onnx.ml並儲存
aoi.opset_import.pop()

print(aoi.opset_import)

import onnx
onnx.save_model(aoi,'/content/drive/MyDrive/aoi.onnx')

[]
